In [1]:
import torch
from brt._C.router import (
    generate_indices_and_loads,
    dispatch_with_indices_and_loads,
)

path_num = 4
gates = torch.randn((8, path_num)).cuda()
topk_indices = torch.topk(gates, k=1, dim=1).indices

hot_mask = (
    torch.zeros(gates.size(0), path_num, dtype=torch.int32, device=gates.device,)
    .scatter_(1, topk_indices, 1)
    .cuda()
)

supported_capacities = torch.Tensor([1, 2, 4, 8, 16]).int().cuda()
# supported_capacities = None

route_indices, dst_loads = generate_indices_and_loads(
    hot_mask, supported_capacities, capacity_padding=True, is_dst_index=True
)
print(route_indices)
print(dst_loads)


/home/whcui/.pyenv/versions/miniconda3-3.8-4.11.0/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 0, 2, 0],
        [0, 0, 0, 2],
        [0, 0, 3, 0],
        [0, 0, 0, 3],
        [0, 2, 0, 0]], device='cuda:0', dtype=torch.int32)
tensor([0, 2, 4, 4], device='cuda:0', dtype=torch.int32)


In [6]:
in_data = torch.randn((8, path_num)).cuda()
print(in_data)
print(route_indices)
print(dst_loads)
out_data_1 = dispatch_with_indices_and_loads(
    in_data, route_indices, dst_loads
)
print(out_data_1)
out_data_1 = dispatch_with_indices_and_loads(
    in_data, route_indices, dst_loads, tag_generating=True
)
print(out_data_1)
out_data_1 = dispatch_with_indices_and_loads(
    in_data, route_indices, dst_loads, max_path_padding=True
)
print(out_data_1)
out_data_1 = dispatch_with_indices_and_loads(
    in_data, route_indices, dst_loads, max_path_padding=True, tag_generating=True
)
print(out_data_1)
# out_data_2 = dispatch_with_dst_indices_1d(in_data, route_indices, dst_loads)
# print(out_data_2)
# torch.allclose(out_data_1[0], out_data_2)


tensor([[-0.2580,  0.5596, -0.1800, -0.3097],
        [ 0.4661, -0.2858,  0.7887, -0.7673],
        [-2.4566,  0.7451, -0.5111, -0.2155],
        [-0.5909, -0.2618, -0.0118,  0.8439],
        [ 0.4452, -0.0725, -1.1200, -1.9552],
        [-1.4211,  0.6426, -0.1254, -1.0632],
        [-0.8001, -0.3336, -0.1789,  0.9407],
        [-0.4520, -1.7542,  1.2423, -0.6588]], device='cuda:0')
tensor([[0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 0, 2, 0],
        [0, 0, 0, 2],
        [0, 0, 3, 0],
        [0, 0, 0, 3],
        [0, 2, 0, 0]], device='cuda:0', dtype=torch.int32)
tensor([0, 2, 4, 4], device='cuda:0', dtype=torch.int32)
[tensor([[-0.2580,  0.5596, -0.1800, -0.3097],
        [-0.4520, -1.7542,  1.2423, -0.6588],
        [ 0.4661, -0.2858,  0.7887, -0.7673],
        [-0.5909, -0.2618, -0.0118,  0.8439],
        [-1.4211,  0.6426, -0.1254, -1.0632],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [-2.4566,  0.7451, -0.5111, -0.2155],
        [ 0.4452, -0.